In [1]:
import pandas as pd
import json
import ast
!pip install fastparquet

# Leitura do arquivo TXT - Utilizando literal_eval para transformações de string para objetos

In [2]:
import ast
  

with open('export_deals.txt',encoding="utf-8") as f:
    data = f.read()

d = ast.literal_eval(data)
  

# Função para organização das hierarquias da estrutura dos dados 

A função conct_dict, foi utilizada para definir a hierarquia dos dados considerando a estrutura de dados no formato dicionário: pois a estrutura correspondente a um registro era representado por um dicionário. No entanto, os dicionários contiam outras estruturas de dados como: dicinários e listas. Está função tem como objetivo extrair os valores destas estruturas aninhadas e padronizar em um dado comum, para isso foi utilizado o nome da chave externa concatenado com nome da chave do interna do dicionário. Com a padronização herdada da modelagem de banco de dados.

Por exemplo: A a estrutura:
{'id': 5373, 'creator_user_id': {'id': 4571337, 'name': 'Bruno Martins', 'email': 'bruno.martins@hsb.com.br'...
Foi transformado para:
{id': 5373, 'creator_user_id_id': 4571337, 'creator_user_id_name': 'Bruno Martins', 'creator_user_id_email': 'bruno.martins@hsb.com.br' ..

In [4]:
def conct_dict (i,dict,name):
    list_name_key =list(d[i][name].keys())
    val_conc = []
    list_values = []
    for key_in_list in list_name_key:
        val_conc.append(f"{name}_{key_in_list}")
    list_values = list(d[i][name].values())
    res = {val_conc[i]: list_values[i] for i in range(len(val_conc))}
    return res 


Para resolução das listas aninhadas contidas nos dicionários a função abaixo foi utilizada para extração dos dados, neste sample de dados foi observado que as listas contiam dicionários e a extração também foi realizada. 

In [5]:
def list_conct_dict (i,dict,name,pos):
    list_name_key =list(d[i][name][pos].keys())
    val_conc = []
    list_values = []
    for key_in_list in list_name_key:
        val_conc.append(f"{name}_{key_in_list}")
    list_values = list(d[i][name][pos].values())
    res = {val_conc[i]: list_values[i] for i in range(len(val_conc))}
    return res 

# Tratamento dos dicionários mais internos

Para extrair os dados foi necessário percorrer cada amostra do conjunto, para verificação da extração foi verificado o tipo de cada chave do dicionário. E se o tipo de dados da chave correspondese a um outro dicionário é realizado a extração dos valores do dicionário mais interno.

In [6]:
#Tratando dicionário
from tqdm import tqdm
lista_aux_del = []
for i in tqdm(range(len(d)),total=len(d)):

    for j in d[i].keys():
        new_values_key = {} 
        if type(d[i][j]) is dict:
            new_values_key = conct_dict(i, d[i][j], j)
            d[i][j] = new_values_key
            lista_aux_del.append(j)

100%|██████████| 1000/1000 [00:00<00:00, 25708.90it/s]


Atualização dos valores obtidos, para o dicionários mais externo:

In [7]:
for i in range (len(d)):
    for j in set(lista_aux_del):
        
        try:
            d[i].update(d[i][j])
            d[i].pop(j)
        except:   
            pass

# Transformação da estrutura de dados dicionário para DataFrame

*   Item da lista
*   Item da lista

In [8]:
df = pd.DataFrame.from_dict(d)

In [9]:
df

,id,stage_id,title,value,currency,add_time,update_time,stage_change_time,active,deleted,...,org_id_cc_email,org_id_value,user_id_id,user_id_name,user_id_email,user_id_has_pic,user_id_pic_hash,user_id_active_flag,user_id_value,person_id
0,5373,80,35172510 - CRESCER loja - (hsb19),0.0,BRL,2018-03-02 16:21:48,2021-10-04 18:40:32,2021-07-15 14:57:12,False,False,...,hsb@pipedrivemail.com,31789,4575434,Ivyla Mendes,ivyla.sousa@hsb.com.br,0,None,True,4575434,NaN
1,5375,80,35102829 - SAO DOMINGOS loja - (hsb19),0.0,BRL,2018-03-02 16:21:49,2021-07-15 14:57:12,2021-07-15 14:57:12,False,False,...,hsb@pipedrivemail.com,32249,7756647,Sarah Vernier,sarah.vernier@hsb.com.br,0,None,False,7756647,NaN
2,5377,80,35178020 - EDUQUE NOVA loja - (hsb19),0.0,BRL,2018-03-02 16:21:49,2021-07-15 14:57:13,2021-07-15 14:57:13,False,False,...,hsb@pipedrivemail.com,31779,7721721,Lucio Neto,lucio.neto@hsb.com.br,1,9f13f013b112c6e1efb6e3794cceb16e,False,7721721,NaN
3,5379,31,31005134 - [loja PARCEIRA18] - loja SAO PAULO ...,76500.0,BRL,2018-03-02 16:21:49,2020-03-24 14:34:16,2019-07-29 14:38:27,False,False,...,hsb@pipedrivemail.com,32685,7721721,Lucio Neto,lucio.neto@hsb.com.br,1,9f13f013b112c6e1efb6e3794cceb16e,False,7721721,NaN
4,5380,80,35105132 - SANTI loja - (hsb19),0.0,BRL,2018-03-02 16:21:49,2021-07-15 14:57:13,2021-07-15 14:57:13,False,False,...,hsb@pipedrivemail.com,32222,6560014,Luma Borges,luma.borges@hsb.com.br,1,337a980d411b06f5cee65c0d08fd54e7,False,6560014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,8038,30,35136566 - MATER ET MAGISTRA CENTRO EDUCACIONA...,69120.0,BRL,2018-05-21 15:08:53,2020-03-24 14:36:36,2019-10-11 13:07:07,False,False,...,hsb@pipedrivemail.com,33648,7756647,Sarah Vernier,sarah.vernier@hsb.com.br,0,None,False,7756647,NaN
996,8040,80,35107273 - ANGLO BRASILEIRO loja - (hsb19),0.0,BRL,2018-05-21 15:08:53,2021-07-15 14:57:47,2021-07-15 14:57:47,False,False,...,hsb@pipedrivemail.com,33650,6560014,Luma Borges,luma.borges@hsb.com.br,1,337a980d411b06f5cee65c0d08fd54e7,False,6560014,NaN
997,8042,80,35158732 - loja ONIS - (hsb19),0.0,BRL,2018-05-21 15:08:53,2021-07-15 14:57:47,2021-07-15 14:57:47,False,False,...,hsb@pipedrivemail.com,33652,8802648,Lidia Araujo,lidia.santos@hsb.com.br,0,None,False,8802648,NaN
998,8044,80,35165347 - MOBILE loja - (hsb19),0.0,BRL,2018-05-21 15:08:54,2021-07-15 14:57:49,2021-07-15 14:57:49,False,False,...,hsb@pipedrivemail.com,33654,8038143,Caroline Lourenço,caroline.lourenco@hsb.com.br,1,34e84171ad459fe8f505e728d2c0d454,False,8038143,NaN


# Exportação do Data Frame para CSV e Parquet formato

In [10]:
df.to_parquet("parquetFile.parquet")
df.to_csv("File_csv.csv")
pd.read_parquet('parquetFile.parquet')

,id,stage_id,title,value,currency,add_time,update_time,stage_change_time,active,deleted,...,org_id_cc_email,org_id_value,user_id_id,user_id_name,user_id_email,user_id_has_pic,user_id_pic_hash,user_id_active_flag,user_id_value,person_id
0,5373,80,35172510 - CRESCER loja - (hsb19),0.0,BRL,2018-03-02 16:21:48,2021-10-04 18:40:32,2021-07-15 14:57:12,False,False,...,hsb@pipedrivemail.com,31789,4575434,Ivyla Mendes,ivyla.sousa@hsb.com.br,0,None,True,4575434,NaN
1,5375,80,35102829 - SAO DOMINGOS loja - (hsb19),0.0,BRL,2018-03-02 16:21:49,2021-07-15 14:57:12,2021-07-15 14:57:12,False,False,...,hsb@pipedrivemail.com,32249,7756647,Sarah Vernier,sarah.vernier@hsb.com.br,0,None,False,7756647,NaN
2,5377,80,35178020 - EDUQUE NOVA loja - (hsb19),0.0,BRL,2018-03-02 16:21:49,2021-07-15 14:57:13,2021-07-15 14:57:13,False,False,...,hsb@pipedrivemail.com,31779,7721721,Lucio Neto,lucio.neto@hsb.com.br,1,9f13f013b112c6e1efb6e3794cceb16e,False,7721721,NaN
3,5379,31,31005134 - [loja PARCEIRA18] - loja SAO PAULO ...,76500.0,BRL,2018-03-02 16:21:49,2020-03-24 14:34:16,2019-07-29 14:38:27,False,False,...,hsb@pipedrivemail.com,32685,7721721,Lucio Neto,lucio.neto@hsb.com.br,1,9f13f013b112c6e1efb6e3794cceb16e,False,7721721,NaN
4,5380,80,35105132 - SANTI loja - (hsb19),0.0,BRL,2018-03-02 16:21:49,2021-07-15 14:57:13,2021-07-15 14:57:13,False,False,...,hsb@pipedrivemail.com,32222,6560014,Luma Borges,luma.borges@hsb.com.br,1,337a980d411b06f5cee65c0d08fd54e7,False,6560014,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,8038,30,35136566 - MATER ET MAGISTRA CENTRO EDUCACIONA...,69120.0,BRL,2018-05-21 15:08:53,2020-03-24 14:36:36,2019-10-11 13:07:07,False,False,...,hsb@pipedrivemail.com,33648,7756647,Sarah Vernier,sarah.vernier@hsb.com.br,0,None,False,7756647,NaN
996,8040,80,35107273 - ANGLO BRASILEIRO loja - (hsb19),0.0,BRL,2018-05-21 15:08:53,2021-07-15 14:57:47,2021-07-15 14:57:47,False,False,...,hsb@pipedrivemail.com,33650,6560014,Luma Borges,luma.borges@hsb.com.br,1,337a980d411b06f5cee65c0d08fd54e7,False,6560014,NaN
997,8042,80,35158732 - loja ONIS - (hsb19),0.0,BRL,2018-05-21 15:08:53,2021-07-15 14:57:47,2021-07-15 14:57:47,False,False,...,hsb@pipedrivemail.com,33652,8802648,Lidia Araujo,lidia.santos@hsb.com.br,0,None,False,8802648,NaN
998,8044,80,35165347 - MOBILE loja - (hsb19),0.0,BRL,2018-05-21 15:08:54,2021-07-15 14:57:49,2021-07-15 14:57:49,False,False,...,hsb@pipedrivemail.com,33654,8038143,Caroline Lourenço,caroline.lourenco@hsb.com.br,1,34e84171ad459fe8f505e728d2c0d454,False,8038143,NaN
